In [ ]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import sklearn
from scipy.signal import savgol_filter
from sklearn import preprocessing
import statistics 
from tqdm import tqdm
import copy
from random import randint 
import math
from matplotlib import ticker
from google.colab import files
from fbprophet import Prophet
from scipy.signal import savgol_filter
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import datetime

In [ ]:
#traindf = pd.read_csv("train.csv")
traindf = pd.read_csv("train_round2.csv")
#testdf = pd.read_csv("test.csv")
testdf = pd.read_csv("faketest_round2.csv")
#full_testdf = pd.read_csv("full_test.csv")
full_testdf = pd.read_csv("full_faketest.csv")
#mobility = pd.read_csv("summarized_mobility.csv")

FileNotFoundError: ignored

### Separate data by state

In [7]:
train_by_state = {}
for state in unique_states:
  train_by_state[state] = traindf[traindf.Province_State == state]

NameError: ignored

## Models

### Linear with Prophet adjusted

In [ ]:
degree = 3
start = 110
alpha = .1
window = 17

#### Weighted average example

In [ ]:
synth_pred = 0.8* np.array(Death_pred[0]).ravel() + 0.2 * forecast.yhat[-(len(sample_test)):].values #simple weighted average
ax = plt.gca()

ax.scatter(np.arange(test_size), sample_test.Deaths.values, label = "Actual")
ax.scatter(np.arange(test_size), Death_pred[0], label = "Linear")
ax.scatter(np.arange(test_size), forecast.yhat.values[-test_size:], label = "Prophet")
ax.scatter(np.arange(test_size), synth_pred, label = "Synthesized")
ax.legend()

plt.plot()

NameError: ignored

#### Ratio example

In [ ]:
### Sythn using daily ratio change
def calc_ratio(data):
  ratios = []
  for i in range(len(data)-1):
    ratios.append(data[i+1]/ data[i])
  return np.array(ratios)

calc_ratio(forecast.yhat.values[-test_size:])

In [ ]:
synth_pred = np.zeros(test_size)
synth_pred[0] = Death_pred[0][0]
synth_pred[1:] = np.array(Death_pred[0]).ravel()[1:] * calc_ratio(forecast.yhat.values[-test_size:])

In [ ]:
#pure ratio adds bias, we will do sth about it below
ax = plt.gca()

ax.scatter(np.arange(test_size), sample_test.Deaths.values, label = "Actual")
ax.scatter(np.arange(test_size), Death_pred[0], label = "Linear")
ax.scatter(np.arange(test_size), forecast.yhat.values[-test_size:], label = "Prophet")
ax.scatter(np.arange(test_size), synth_pred, label = "Synthesized")
ax.legend()

plt.plot()

In [ ]:
synth_pred -= 2 * (np.mean(synth_pred) - np.mean(Death_pred[0]))

In [ ]:
ax = plt.gca()

ax.scatter(np.arange(test_size), sample_test.Deaths.values, label = "Actual")
ax.scatter(np.arange(test_size), Death_pred[0], label = "Linear")
#ax.scatter(np.arange(test_size), forecast.yhat.values[-test_size:], label = "Prophet")
ax.scatter(np.arange(test_size), synth_pred, label = "Synthesized")
ax.legend()

plt.plot()

#### Evaluating weighted avg, ratio with previous linear

In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
MAPE(synth_pred, sample_test.Deaths)

In [ ]:
# All model performances
test_result = {}
feature = 'Deaths'

for state in unique_states:

  ######=========================================######
  # Linear model
  Death_pred = []
  sample_train = train_by_state[state]
  sample_test = testdf[testdf.Province_State == state]
  test_size = len(sample_test)

  ds = sample_train
  value = ds[feature].values
  value = savgol_filter(value.reshape(-1), window, degree)
  combine = [[value[i]] for i in range(len(value))]

  y = np.array([value[start:]]).reshape(-1, 1)
  x = [[i + start] for i in range(142-start)]

  model = Pipeline([
      ("poly", PolynomialFeatures(degree=2)),
      ("lasso_reg", Ridge(alpha=alpha))
  ])
  model.fit(x, y)

  x_test = [[i + 142] for i in range(test_size)]

  y_hat = model.predict(x_test)
  Death_pred.append(y_hat)

  ######=========================================######
  # Prophet
  basic_model = Prophet()
  basic_model.fit(sample_train.iloc[-20:].reset_index() \
              .rename(columns={'Date':'ds',
                               'Deaths':'y'}))

  future = basic_model.make_future_dataframe(periods=test_size)
  forecast = basic_model.predict(future)


  ######=========================================######
  # Synthesized
  avrg_synth  = 0.8* np.array(Death_pred[0]).ravel() + 0.2 * forecast.yhat[-(len(sample_test)):].values

  calc_ratio(forecast.yhat.values[-test_size:])
  ratio_synth = np.zeros(test_size)
  ratio_synth[0] = Death_pred[0][0]
  ratio_synth[1:] = np.array(Death_pred[0]).ravel()[1:] * calc_ratio(forecast.yhat.values[-test_size:])

  ######=========================================######
  # Eval and save result
  test_result[state] = {}
  lin_MAPE = MAPE(Death_pred[0], sample_test.Deaths)
  test_result[state]["linear_Eval"] = lin_MAPE
  avg_MAPE = MAPE(avrg_synth, sample_test.Deaths)
  test_result[state]["W_avrg_Eval"] = avg_MAPE
  ratio_MAPE = MAPE(ratio_synth, sample_test.Deaths)
  test_result[state]["Ratio_Eval"] = ratio_MAPE
  test_result['avr_pred'] = avrg_synth
  test_result['ratio_pred'] = avrg_synth
  test_result['linear'] = Death_pred[0]

  print("state: {}, linear: {}, avg: {}, ratio:{}".format(state, lin_MAPE, avg_MAPE, ratio_MAPE))
  print("===========")



In [ ]:
max_avg_MAPE = -1
max_ratio_MAPE = -1
better_before = {}

for state in unique_states:
  state_data = test_result[state]
  prev = state_data["linear_Eval"]
  avg = state_data["W_avrg_Eval"]
  ratio =  state_data["Ratio_Eval"]

  if avg > max_avg_MAPE:
    max_avg_MAPE = avg
  
  if avg > max_ratio_MAPE:
    max_ratio_MAPE = ratio

  # MAPE so the smaller the better
  if avg < prev  or ratio < prev:
    if ratio < avg:
      better_before[state] = ("ratio", prev, ratio)
    else:
      better_before[state] = ("avg", prev, avg)

(max_avg_MAPE, max_ratio_MAPE)

In [ ]:
len(set(better_before.keys()))

In [ ]:
test_result

### Prophet
#### links: https://facebook.github.io/prophet/docs/quick_start.html. 
#### https://peerj.com/preprints/3190.pdf

In [ ]:
sample_train = train_by_state["California"]
sample_test = testdf[testdf.Province_State == "California"]

In [ ]:
sample_train = sample_train.reset_index(drop= True)
sample_train.head()

In [ ]:
# California
plt.plot(np.arange(142), sample_train.Deaths.values, label = "train")
plt.plot(np.arange(142, 168), sample_test.Deaths.values, label = "test")
plt.legend()
plt.show()

In [ ]:
# Alaska
plt.plot(np.arange(142), sample_train.Deaths.values, label = "train")
plt.plot(np.arange(142, 168), sample_test.Deaths.values, label = "test")
plt.legend()
plt.show()

In [ ]:
basic_model = Prophet()
basic_model.fit(sample_train.reset_index() \
              .rename(columns={'Date':'ds',
                               'Deaths':'y'}))

In [ ]:
future = basic_model.make_future_dataframe(periods=len(sample_test))
future #contains all dates of train and test

In [ ]:
forecast = basic_model.predict(future)
forecast.tail()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(np.arange(142), sample_train.Deaths.values, label = "train")
ax1.plot(np.arange(142, 168), sample_test.Deaths.values, label = "test")
ax1.plot(np.arange(142, 168), forecast.yhat.values[142:], label = "forecast")
ax1.legend()

ax2.scatter(np.arange(26), sample_test.Deaths.values, label = "test")
ax2.scatter(np.arange(26), forecast.yhat.values[142:], label = "forecast")
ax2.legend()

fig.set_size_inches(11, 4)
plt.show()

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = basic_model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), basic_model, forecast)

In [ ]:
basic_model.changepoints

In [ ]:
components_plot = basic_model.plot_components(forecast)

#### Example of prophet

In [8]:
basic_model = Prophet()
basic_model.fit(sample_train.iloc[-20:].reset_index() \
              .rename(columns={'Date':'ds',
                               'Deaths':'y'}))

future = basic_model.make_future_dataframe(periods=len(sample_test))
forecast = basic_model.predict(future)
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(np.arange(20), sample_train.iloc[-20:].Deaths.values, label = "train")
ax1.plot(np.arange(20, 46), sample_test.Deaths.values, label = "test")
ax1.plot(np.arange(20, 46), forecast.yhat.values[20:], label = "forecast")
ax1.legend()

ax2.scatter(np.arange(26), sample_test.Deaths.values, label = "test")
ax2.scatter(np.arange(26), forecast.yhat.values[20:], label = "forecast")
ax2.legend()

fig.set_size_inches(11, 4)
plt.show()

NameError: ignored

In [ ]:
fig = basic_model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), basic_model, forecast)

### Prophet with additional regressors

In [ ]:
sample_train.head()

In [ ]:
sample_train.columns

In [ ]:
X_train = sample_train.iloc[:, 5:6]
X_train['Incident_Rate'] = sample_train['Incident_Rate']
X_train['People_Tested'] = sample_train['People_Tested']
X_train['People_Hospitalized'] = sample_train['People_Hospitalized']
X_train['Confirmed'] = sample_train['Confirmed']
X_train['Deaths'] = sample_train["Deaths"]
X_train['Mobility'] = sample_train["Mobility"]
X_train.head()

In [ ]:
future

In [ ]:
feature_future = future.copy()

In [ ]:
for feature in ['Incident_Rate', 'People_Tested', 'Confirmed', 'Mobility']:
  feature_model = Prophet()
  feature_model.fit(sample_train.reset_index() \
              .rename(columns={'Date':'ds',
                               feature:'y'}))

  forecast = feature_model.predict(feature_future)
  future[feature] = forecast.yhat
  feature_model.plot(forecast)
  plt.title(f"{feature} modeled")
  plt.show()

In [ ]:
pro_regressor= Prophet()
pro_regressor.add_regressor('Incident_Rate')
pro_regressor.add_regressor('People_Tested')
#pro_regressor.add_regressor('People_Hospitalized')
pro_regressor.add_regressor('Confirmed')
pro_regressor.add_regressor('Mobility')

pro_regressor.fit(X_train.reset_index(drop= True) \
              .rename(columns={'Date':'ds',
                               'Deaths':'y'}))

forecast = pro_regressor.predict(future)
result = pro_regressor.plot(forecast)

In [ ]:
len(forecast.yhat.values)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(np.arange(142), sample_train.Deaths.values, label = "train")
ax1.plot(np.arange(142, 168), sample_test.Deaths.values, label = "test")
ax1.plot(np.arange(142, 168), forecast.yhat.values[-26:], label = "forecast")
ax1.legend()

ax2.scatter(np.arange(26), sample_test.Deaths.values, label = "test")
ax2.scatter(np.arange(26), forecast.yhat.values[-26:], label = "forecast")
ax2.legend()

fig.set_size_inches(11, 4)
plt.show()